In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import logging

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout
import numpy as np

from art.attacks import DeepFool
from art.attacks import FastGradientMethod
from art.classifiers import KerasClassifier
from art.utils import load_dataset

from keras import backend as K
K.clear_session()
from keras.models import load_model

from keras.applications.vgg16 import (VGG16, preprocess_input, decode_predictions)
import cv2
from keras.preprocessing import image

import cv2
import scipy.misc
from PIL import Image
import imageio

E:\software\Anaconda3\envs\python3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
E:\software\Anaconda3\envs\python3\lib\site-packages\tensorflow\python\framework\dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
E:\software\Anaconda3\envs\python3\lib\site-packages\tensorflow\python\framework\dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
E:\software\Anaconda3\envs\python3\lib\si

In [2]:
# Step 1:  Read dataset

x_test = np.load('./ImageNet/val_500.npy')
y_test = np.load('./ImageNet/y_val_500.npy')
min_, max_ = 0,255


In [3]:
x_test = x_test[0:100]
y_test = y_test[0:100]

In [4]:
# Step 2: Create the model

model = VGG16(weights='imagenet')

In [5]:
# Step 3: Create the ART classifier

classifier = KerasClassifier(model=model, clip_values=(min_, max_), use_logits=False)

In [6]:
# Step 4: Evaluate the ART classifier on benign test examples

predictions = classifier.predict(x_test)
print (predictions.shape)

(100, 1000)


In [14]:
#print (np.argmax(predictions, axis=1))
cmp = (np.argmax(predictions, axis=1))
truthIndex = []
for i in range(100):
    if cmp[i] == y_test[i]:
        truthIndex.append(i)
print (truthIndex)
print (len(truthIndex))

[0, 8, 9, 10, 11, 12, 13, 14, 15, 19, 20, 22, 23, 25, 26, 28, 30, 31, 34, 35, 41, 43, 46, 51, 52, 53, 55, 62, 63, 65, 66, 68, 71, 72, 73, 74, 75, 77, 80, 81, 82, 85, 88, 89, 92, 94, 96, 97, 98, 99]
50


In [8]:
accuracy = np.sum(np.argmax(predictions, axis=1) == y_test) / len(y_test)
print('Accuracy on benign test examples: {}%'.format(accuracy * 100))

Accuracy on benign test examples: 50.0%


In [9]:
# Step 6: Generate adversarial test examples
attack = FastGradientMethod(classifier=classifier, eps=0.2)
x_test_adv = attack.generate(x=x_test)

In [10]:
# Step 7: Evaluate the ART classifier on adversarial test examples

predictions = classifier.predict(x_test_adv)
accuracy = np.sum(np.argmax(predictions, axis=1) == y_test) / len(y_test)
print('Accuracy on adversarial test examples: {}%'.format(accuracy * 100))

Accuracy on adversarial test examples: 37.0%
